In [1]:
# Creating scatterplot graph comparing Flight Passengers between City Pairs vs. Distance

import pandas as pd
import openpyxl
import plotly.express as px
import plotly.graph_objects as go

In [2]:
file_path = 'https://raw.githubusercontent.com/Davideij/Python-Project-Group-3/main/Data/CITIES_FINAL.xlsx'
df = pd.read_excel(file_path)
df.head()

,City_A_Name,City_B_Name,City_A,City_B,Total passengers,NUTS_3_code_A,NUTS_3_code_B,Distance,has_connection
0,Paris,Toulouse,LFPO,LFBO,4716804,FR107,FRJ23,610672.306409,0
1,Barcelona,Madrid,LEBL,LEMD,4657590,ES511,ES300,496744.786620,1
2,Nice,Paris,LFMN,LFPO,4249404,FRL03,FR107,645376.754517,0
3,Berlin,Munich,EDDT,EDDM,3878625,DE300,DE212,501031.167189,0
4,Berlin,Frankfurt,EDDT,EDDF,3869795,DE300,DE712,423686.712363,0


In [3]:
# Creating list to store yearly dataFrames
sheets = ['2016', '2017', '2018', '2019']
data_frames = []

# Loop through each sheet, add a 'Year' column, and append it to data_frames list
for sheet in sheets:
    df = pd.read_excel(file_path, sheet_name=sheet)
    df['Year'] = sheet  # Add a Year column for each DataFrame
    data_frames.append(df)

# Concatenate all data into a single DataFrame
df_all = pd.concat(data_frames, ignore_index=True)

# Concatenate all data into a single DataFrame
df_all = pd.concat(data_frames, ignore_index=True)

# Convert Distance from meters to kilometers
df_all['Distance'] = df_all['Distance'] / 1000

# Filtering city pairs based on the Distance range (150 to 1200 km)
df_all = df_all[(df_all['Distance'] >= 150) & (df_all['Distance'] <= 1200)]

# Extract relevant columns using the actual column names in the file
df_all = df_all[['Total passengers', 'NUTS_3_code_A', 'NUTS_3_code_B', 'Distance', 'Year', 'City_A_Name', 'City_B_Name', 'has_connection']]

# Combining city names for the custom hover name
df_all['City_Pair'] = df_all['City_A_Name'] + " & " + df_all['City_B_Name']

# Converts "has_connection" column to string for plotly express will see it as a binary option (not continuous
df_all['has_connection'] = df_all['has_connection'].astype(str)

# Filter to Top 50 Total passenger volumes
df_all = df_all.sort_values(by='Total passengers', ascending=False).groupby('Year').head(100)

# Sorting year slider in ascending order, aka making sure the slider is correctly laid out
df_all = df_all.sort_values(by=['Year', 'Total passengers'], ascending=[True, False])

# Map the 'has_connection' column to specific colors (why isn't the red showing up??)
color_map = {'0': 'blue', '1': '#F31F1F'} # Medium-dark red for 1

# Create a scatter plot using Plotly Express
fig = px.scatter(
    df_all,
    x='Distance',
    y='Total passengers',
    color = 'has_connection',
    color_discrete_map = color_map,
    animation_frame='Year',
    hover_name='City_Pair',
   hover_data={'NUTS_3_code_A': True, 'NUTS_3_code_B': True, 'Distance': True, 'Total passengers': True, 'Year': False,'City_A_Name': False, 'City_B_Name': False},
    title="Air Passenger Volumes vs Distance Between City Pairs (2016-2019)",
    labels={"Distance": "Distance (km)", "Total passengers": "Total Yearly Passengers"},
)

# Update hover template to show City Pair with custom text
fig.update_traces(hovertemplate="<br>".join([
    "City Pair: %{hovertext}",
    "Distance: %{x} km",
    "Passengers: %{y}"
]))
# Adds title for legend
fig.update_layout(legend_title_text='HSR Connection Status')

# Update legend to rename "0" to "No pre-existing connection" and "1" to "Already connected"
fig.for_each_trace(lambda t: t.update(name="No pre-existing connection" if t.name == "0" else "Already connected"))


fig.show()

In [4]:
# Create a dictionary to store the top 10 tables for each year
top_10_tables = {}

# Looping through each year, filter the data, and get the top 10 rows
for year in sheets:  # 'sheets' contains ['2016', '2017', '2018', '2019']
    # Filter for the specific year and sort by 'Total Passengers' in descending order
    top_10 = df_all[df_all['Year'] == year].sort_values(by='Total passengers', ascending=False).head(10)
    # Store the result in the dictionary
    top_10_tables[year] = top_10[['City_Pair', 'Total passengers', 'Distance', 'has_connection']]

# Display the tables
for year, table in top_10_tables.items():
    print(f"\nTop 10 City Pairs with Highest Air Passenger Volume in {year}")
    print(table.to_string(index=False))

# Note: distance is in kilometers


Top 10 City Pairs with Highest Air Passenger Volume in 2016
                    City_Pair  Total passengers   Distance has_connection
             Paris & Toulouse           4716804 610.672306              0
           Barcelona & Madrid           4657590 496.744787              1
                 Nice & Paris           4249404 645.376755              0
              Berlin & Munich           3878625 501.031167              0
           Berlin & Frankfurt           3869795 423.686712              0
             Hamburg & Munich           3603238 608.827546              0
Barcelona & Palma de Mallorca           3545242 249.225542              0
   Madrid & Palma de Mallorca           3375136 576.185840              0
         DÃ¼sseldorf & Munich           3128381 483.061898              0
          Frankfurt & Hamburg           2742869 392.608855              0

Top 10 City Pairs with Highest Air Passenger Volume in 2017
                    City_Pair  Total passengers   Distance has_c

In [5]:
top_10_tables["2016"].loc[:, "Year"] = "2016"
top_10_tables["2017"].loc[:, "Year"] = "2017"
top_10_tables["2018"].loc[:, "Year"] = "2018"
top_10_tables["2019"].loc[:, "Year"] = "2019"

# Concatenate the DataFrames
df_combined = pd.concat([top_10_tables["2016"], top_10_tables["2017"], top_10_tables["2018"], top_10_tables["2019"]])

# Step 2: Identify the top 10 city pairs based on total passengers across all years
top_city_pairs = df_combined.groupby('City_Pair')['Total passengers'].sum().nlargest(10).index

# Step 3: Filter the DataFrame to include only the top 10 city pairs
df_filtered = df_combined[df_combined['City_Pair'].isin(top_city_pairs)]

# Step 4: Initialize an empty figure
fig = go.Figure()

# Plot data for each year and each city pair
years = ['2019', '2018', '2017', '2016']
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']  # Color palette for bars by year

for year, color in zip(years, colors):
    # Filter data for the specific year
    df_year = df_filtered[df_filtered['Year'] == year]
    
    # Add bar trace for each year
    fig.add_trace(go.Bar(
        x=df_year['City_Pair'],
        y=df_year['Total passengers'],
        name=str(year),
        marker_color=color
    ))

# Step 5: Update layout to make the chart more readable
fig.update_layout(
    barmode='group',
    title="Passenger Numbers by City Pair and Year",
    xaxis_title="City Pairs",
    yaxis_title="Total Passengers",
    xaxis={'categoryorder': 'total descending'},
    legend_title="Year",
    width=900,
    height=600
    
)
    
# Show the figure
fig.show()


C:\Users\yanni\AppData\Local\Temp\ipykernel_4752\2278175640.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
#Remove not-applicable city pairs
remove = ["Barcelona & Madrid",            #Already has a HSR connection
          "Barcelona & Palma de Mallorca", #Assumed not feasible to connect an island via HSR
          "Madrid & Palma de Mallorca"     #Assumed not feasible to connect an island via HSR
         ]

# Filter rows based on the City_Pair column and drop them by index
df_combined = df_combined[~df_combined["City_Pair"].isin(remove)]

In [7]:
# Identify the top 10 city pairs based on total passengers across all years
top_city_pairs = df_combined.groupby('City_Pair')['Total passengers'].sum().nlargest(10).index

# Filter the DataFrame to include only the top 10 city pairs
df_filtered = df_combined[df_combined['City_Pair'].isin(top_city_pairs)]

# Initialize an empty figure
fig = go.Figure()

# Plot data for each year and each city pair
years = ['2019', '2018', '2017', '2016']
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']  # Color palette for bars by year

for year, color in zip(years, colors):
    # Filter data for the specific year
    df_year = df_filtered[df_filtered['Year'] == year]
    
    # Add bar trace for each year
    fig.add_trace(go.Bar(
        x=df_year['City_Pair'],
        y=df_year['Total passengers'],
        name=str(year),
        marker_color=color
    ))

# Update layout to make the chart more readable
fig.update_layout(
    barmode='group',
    title="Passenger Numbers by City Pair and Year",
    xaxis_title="City Pairs",
    yaxis_title="Total Passengers",
    xaxis={'categoryorder': 'total descending'},
    legend_title="Year",
    width=900,
    height=600
    
)
    
# Show the figure
fig.show()